# Imports

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import xarray as xr
import yaml
import warnings
from datetime import datetime as dt
from datetime import timedelta as td
from itertools import cycle
from pathlib import Path
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib import colormaps
from matplotlib.patches import Rectangle
from hbt_tools import plots as hp
from hbt_tools import utils_hbt as uh

# Functions

In [ ]:
def open_ds(file_path):
    with xr.open_dataset(file_path) as ds:
        return ds
#-----------------------------------------------------------------------------------------------------------------------

def add_geo_features(ax):
    """
    Add reference geographical layers such as thenational boundaries
    and the main lakes and rivers.
    """
    ax.add_feature(
        cfeature.NaturalEarthFeature(
            "cultural",
            "admin_0_boundary_lines_land",
            scale="10m",
            edgecolor="black",
            facecolor="none",
            linewidth=1,
        )
    )
    ax.add_feature(
        cfeature.NaturalEarthFeature(
            "physical",
            "rivers_lake_centerlines",
            scale="10m",
            edgecolor=np.array([0.59375, 0.71484375, 0.8828125]),
            facecolor="none",
            linewidth=1,
        )
    )
#-----------------------------------------------------------------------------------------------------------------------

def read_yaml(file_path):
    with open(file_path) as f:
        data_dict = yaml.load(f, Loader=yaml.FullLoader)
    
    return data_dict
#-----------------------------------------------------------------------------------------------------------------------

def plot_precipfield(ds, save_path, figsize=(12, 7), fps=4):
    epsg_lv03 = 21781
    crs = ccrs.epsg(epsg_lv03)
    dt_format = '%d.%m.%Y %H:%M'

    # Plot settings
    levels = np.logspace(-1, 2, 10, base=10)
    cmap = "Blues"
    fig = plt.figure(figsize=figsize);

    precip_field = ds.RR.isel(time=0)
    precip_field = precip_field.where(precip_field > 0)
    dt0_str = ds.time.to_pandas().iloc[0].strftime(dt_format)

    g = precip_field.plot(
        transform=crs,
        subplot_kws=dict(projection=crs),
        levels=levels,
        cmap=cmap
        )
    ax = g.axes
    title = ax.title
    add_geo_features(ax);

    ax.add_patch(Rectangle((600000, 190000), 2e4, 2e4, edgecolor='red', fill=False));

    # Animation
    def animate(t):
        data = ds.RR.isel(time=t)
        time = ds.time.to_pandas().iloc[t] # time as Timestamp()
        dt_str = time.strftime(dt_format)
        data = data.where(data > 0).data.flatten()
        g.set_array(data)
        title.set_text(dt_str)
        return (g, title) # must be given as tuple

    def init():
        data = precip_field.data.flatten()
        g.set_array(data)
        title.set_text(dt0_str)
        return (g, title) # must be given as tuple

    ani = FuncAnimation(fig, animate, range(len(ds.time)), init_func=init)
    ani.save(save_path, writer=PillowWriter(fps=fps))
#-----------------------------------------------------------------------------------------------------------------------

def xarrays2ts(nc_list, dict_coords):
    ts_list = []
    for file in nc_list:
        with xr.open_dataset(file) as ds:
            ts =  pd.DataFrame(columns=['time', *dict_coords.keys()])
            for k, v in dict_coords.items():
                ts[k] = ds.sel(chx=v[0], chy=v[1], method='nearest').RR.data
            
            ts['time'] = ds.time.data
            ts_list.append(ts)

    ts_out = pd.concat(ts_list)
    ts_out.set_index(keys='time', inplace=True)

    ts_out = ts_out / 60 * 10 # Convert units: mm/h -> mm/10min
    ts_out = ts_out.round(1)

    return ts_out
#-----------------------------------------------------------------------------------------------------------------------

def ncs_to_ds(nc_list, slice_x, slice_y, dim='time'):
    ds_list = []
    for file in nc_list:
        with xr.open_dataset(file) as ds:
            ds = ds.sel(chx=slice_x, chy=slice_y, method=None)
            ds_list.append(ds)

    ds_out = xr.concat(ds_list, dim=dim)

    return ds_out
#-----------------------------------------------------------------------------------------------------------------------

def obsname_from_date(date):
    date = date.strftime('%Y%m')
    obs_name = f"radar-observations_{date}.nc"

    return obs_name
#-----------------------------------------------------------------------------------------------------------------------

def fcstname_from_date(date):
    date = date.strftime('%Y%m%d%H%M')
    fcst_name = f"RR_INCA_{date}.nc"

    return fcst_name
#-----------------------------------------------------------------------------------------------------------------------

def ts_to_cumulativ(ts):
    ts_out = ts.copy()
    for name, col in ts_out.items():
        ts_out[name] = col.cumsum()

    return ts_out
#-----------------------------------------------------------------------------------------------------------------------

def convert_ds_units(ds, factor, variable, new_units):
    ds = ds.copy(deep=True)
    ds[variable] = ds[variable] * factor
    ds[variable].attrs.update({'units': new_units})

    return ds
#-----------------------------------------------------------------------------------------------------------------------

def compute_ds_diff(ds_obs, ds_fcst, variable, method='abs'):
    '''
    Computes the difference between forecast (fcst) and observations (obs).

    The difference is computed as fcst - obs. This means, that positive values
    indicate that forecast overestimates the reality, whereas negative values
    indicate the opposite, that is forecast underestimates the reality.
    '''
    ds_obs = ds_obs.copy(deep=True)
    ds_fcst = ds_fcst.copy(deep=True)

    ds_obs = ds_obs[variable]
    ds_fcst = ds_fcst[variable]

    time_start = ds_fcst.time[0]
    time_end = ds_fcst.time[-1]
    time_slice = slice(time_start, time_end)

    time_fcst = ds_fcst.time.sel(time=time_slice)
    time_obs = ds_obs.time.sel(time=time_slice)

    # BUG: diff(time_fcst/time_obs) != diff(time_obs/time_fcst)
    if len(time_fcst > time_obs):
        time_diff = set(time_fcst.to_numpy()).difference(set(time_obs.to_numpy()))
    else:
        time_diff = set(time_obs.to_numpy()).difference(set(time_fcst.to_numpy()))

    if time_diff:
        time_diff = list(time_diff) # cannot drop using set
        warnings.warn(f"The index 'time' for 'ds_obs' and 'ds_fcst' differs on "
            f"the following elements: {time_diff}")
        time_fcst = time_fcst.drop_sel(time=time_diff)

    ds_obs = ds_obs.sel(time=time_fcst)
    ds_fcst = ds_fcst.sel(time=time_fcst)

    if method == 'abs':
        ds_out = ds_fcst - ds_obs

    elif method == 'rel':
        # ds_obs / ds_fcst is often 0 -> alternative: RPD - Relative Percent Difference
        ds_out = 2 * (ds_fcst - ds_obs) / (abs(ds_obs) + abs(ds_fcst))
        t_steps = len(ds_out.time)
        for t in range(t_steps):
            data = ds_out.isel(time=t).data
            data[np.isnan(data)] = 0.0
            ds_out.isel(time=t).data = data

    return ds_out
#-----------------------------------------------------------------------------------------------------------------------

def xarray_to_gif(ds, save_path, variable=None, figsize=(7,7), levels=None, cmap=None, datum_format='%d.%m.%Y %H:%M', fps=4):
    ds = ds.copy(deep=True)

    epsg_lv03 = 21781
    crs = ccrs.epsg(epsg_lv03)

    # Plot settings
    fig = plt.figure(figsize=figsize);

    if variable:
        ds = ds[variable]

    if levels is None:
        max_abs = abs(ds).max()
        levels = np.linspace(-max_abs, max_abs, 21)

    if cmap is None:
        cmap = colormaps['RdBu']


    raster_field = ds.isel(time=0)
    datum0_str = ds.time.to_pandas().iloc[0].strftime(datum_format)

    g = raster_field.plot(
        transform=crs,
        subplot_kws=dict(projection=crs),
        levels=levels,
        cmap=cmap,
    )

    ax = g.axes
    title = ax.title

    # Animation
    def animate(t):
        data = ds.isel(time=t)
        time = ds.time.to_pandas().iloc[t] # time as Timestamp()
        datum_str = time.strftime(datum_format)
        g.set_array(data)
        title.set_text(datum_str)

        return (g, title) # must be given as tuple

    def init():
        data = raster_field.data.flatten()
        g.set_array(data)
        title.set_text(datum0_str)

        return (g, title) # must be given as tuple

    ani = FuncAnimation(fig, animate, range(len(ds.time)), init_func=init)
    ani.save(save_path, writer=PillowWriter(fps=fps))
#-----------------------------------------------------------------------------------------------------------------------

def sum_per_timestep(ds, factor):
    time = ds.time.data
    res_list = []

    for t in time:
        ds_sel = ds.sel(time=t)
        res = ds_sel.sum().data * factor
        res = pd.DataFrame([[t, res]], columns=['time', 'value'])
        res_list.append(res)

    df_res = pd.concat(res_list)
    df_res.set_index('time', inplace=True)

    return df_res
#-----------------------------------------------------------------------------------------------------------------------


# Parameters

In [ ]:
dir_radarobs = Path(r"C:\Users\sru\Downloads\Radarbeobachtungen_Meteoschweiz")
dir_radarfcst = Path(r"H:\8 Archiv\Radardaten_Meteoschweiz")

obs_list = list(dir_radarobs.glob('*.nc'))
fcst_list = list(dir_radarfcst.glob('*.nc'))

save_dir = Path(r"H:\2 Projekte\1000-\1200-\1296\1296.29 Integrale Bewirtschaftung Phase 3\05 Berechnungen Grundlagen"
    r"\02 - Validierung Niederschlagsradardaten\Plots")

yaml_events = Path(r"H:\2 Projekte\1000-\1200-\1296\1296.29 Integrale Bewirtschaftung Phase 3"
    r"\05 Berechnungen Grundlagen\02 - Validierung Niederschlagsradardaten\Regenereignisse.yaml")
yaml_stations = Path(r"H:\2 Projekte\1000-\1200-\1296\1296.29 Integrale Bewirtschaftung Phase 3"
    r"\05 Berechnungen Grundlagen\02 - Validierung Niederschlagsradardaten\stations.yaml")

start_date = '01.12.2022'
end_date = '31.12.2023'
date_format = '%d.%m.%Y %H:%M'

mmh_mm10min = 10/60
id_BER = 2561

coord_BER = (601934.00, 204410.00)
region_worblental = [
    (600000, 210000),
    (620000, 210000),
    (620000, 190000),
    (600000, 190000)
]
slice_x = slice(600000, 620000)
slice_y = slice(190000, 210000)

# Read Inputs

In [ ]:
# Read yaml files
events = read_yaml(yaml_events)
stations = read_yaml(yaml_stations)

In [ ]:
# Read data BER
ts_BER = uh.mdb_getdata(id_BER, start_date, end_date, date_format='%d.%m.%Y')
ts_BER.index = ts_BER.index.tz_convert('utc')

In [ ]:
# Read all obs
ds_obs = ncs_to_ds(nc_list=obs_list, slice_x=slice_x, slice_y=slice_y)
ds_obs = convert_ds_units(ds_obs, factor=mmh_mm10min, variable='RR', new_units='mm/10min')
ds_obs = ds_obs.round(1)

In [ ]:
# Check ds_obs
time_index = ds_obs.get_index(key='time')
time_index

# Check duplicated
duplicates = time_index[time_index.duplicated()]
duplicates

# Remove duplicates
ds_obs = ds_obs.drop_duplicates(dim='time')

In [ ]:

ts_stations = xarrays2ts(obs_list, stations)

# Analysis

## Comparison Bodenmessung / Radarmessung

In [ ]:
ts_Zollikofen = pd.DataFrame(ts_stations.loc[:,'Zollikofen'])
save_path = Path(r"H:\2 Projekte\1000-\1200-\1296\1296.29 Integrale Bewirtschaftung Phase 3\05 Berechnungen Grundlagen"
    r"\03 - Regenereignisse\Niederschlag_01_Ereignisdetektion\Radarmessung_Zollikofen.csv")
ts_Zollikofen.to_csv(save_path, sep=';', date_format=date_format)

save_path = save_dir / 'Vergleich_Boden-Radarmessung.html'
lines = [
    dict(color='blue'),
    dict(color='red'),
    dict(color='blue', dash='dash'),
    dict(color='red', dash='dash')
]
fig = hp.ply_2y([ts_BER, ts_Zollikofen], [ts_BER.cumsum(), ts_Zollikofen.cumsum()],
    names=['Bodenmessung BER', 'Radarmessung Zollikofen', 'Bodenmessung kumulativ', 'Radarmessung kumulativ'],
    xlabel='Zeit', ylabels=['Regen [mm/10min]', 'Regen kumulativ [mm]'],
    title='Vergleich Bodenmessung / Radarmessung', lines=lines, save_path=save_path)

## Radarmessung: Comparison different Stations

In [ ]:
# Stations timeseries
save_path = save_dir / 'Vergleich_TS_Stationen.html'
fig = hp.ply_1y(ts_stations, names=ts_stations.columns, xlabel='Zeit', ylabel='Regen [mm/10min]',
    title='Regenzeitreihen Stationen', save_path=save_path)

In [ ]:
# Stations cumulative sum
ts_stations_cum = ts_to_cumulativ(ts_stations)

save_path = save_dir / 'Vergleich_TS-cum_Stationen.html'
fig = hp.ply_1y(ts_stations_cum, names=ts_stations.columns, xlabel='Zeit', ylabel='Regen kumulativ [mm]',
    title='Regensummen Stationen', save_path=save_path)

## Comparison OBS - FCST

### Plot Schweiz

In [ ]:
date_format = '%d.%m.%Y %H:%M'
ts_events = {}

for event, date in events.items():
    out_dir = save_dir / event
    if not out_dir.exists():
        out_dir.mkdir()

    start_date = dt.strptime(date, date_format)
    end_date = start_date + td(hours=6)

    yearmonth = start_date.strftime('%Y%m')
    file_name = f'radar-observations_{yearmonth}.nc'
    file_path = dir_radarobs / file_name

    with xr.open_dataset(file_path) as ds:
        ds = ds.sel(time=slice(start_date, end_date))

    date_str = start_date.strftime('%y%m%d_%H%M')
    save_path = out_dir / f'Schweiz_{date_str}.gif'
    plot_precipfield(ds, save_path);


    file_name = fcstname_from_date(start_date)
    file_path = dir_radarfcst / file_name

    if file_path.exists():
        with xr.open_dataset(file_path) as ds_fcst:
            ts_fcst =  pd.DataFrame(columns=['time', *stations.keys()])
            for name, coords in stations.items():
                ts_fcst[name] = ds_fcst.sel(chx=coords[0], chy=coords[1], method='nearest').RR.data

            ts_fcst['time'] = ds_fcst.time.data
            ts_fcst.set_index('time', inplace=True)
            ts_fcst = ts_fcst / 60 * 10 # Convert units: mm/h -> mm/10min
            ts_fcst = ts_fcst.round(1)
            ts_fcst.columns = [f"FCST {x}" for x in ts_fcst.columns]
        
            save_path = out_dir / f'FCST-Stations_{date_str}.csv'
            ts_fcst.to_csv(save_path, sep=';')

            ts_events.update({event: ts_fcst})

### Plot Worblental

In [ ]:
for event, date in events.items():
    start_date = dt.strptime(date, date_format)
    end_date = start_date + td(hours=6)

    ds = ds_obs.sel(time=slice(start_date, end_date))

    save_path = save_dir / event /  f"Worblental_{start_date.strftime('%y%m%d_%H%M')}.gif"
    plot_precipfield(ds, save_path, figsize=(8, 8))

### Comparison: Stations Timeseries

In [ ]:
colors = cycle(px.colors.qualitative.Plotly)
for event, ts_fcst in ts_events.items():
    start_date = ts_fcst.index[0]
    end_date = ts_fcst.index[-1]

    ts_obs = ts_stations.loc[start_date:end_date]
    ts_obs.columns = [f"OBS {x}" for x in ts_obs.columns]

    fig = go.Figure()
    for i in range(ts_obs.shape[1]):
        color = next(colors)
        fig.add_trace(go.Scatter(x=ts_obs.index, y=ts_obs.iloc[:,i], name=ts_obs.columns[i], line=dict(color=color)))
        fig.add_trace(go.Scatter(x=ts_fcst.index, y=ts_fcst.iloc[:,i], name=ts_fcst.columns[i], line=dict(dash='dash', color=color)))

        fig.update_xaxes(title='Zeit')
        fig.update_yaxes(title="Regen [mm/10min]")
        fig.update_layout(title=event)

        save_path = save_dir / event / 'Comparison_obs_fcst.html'
        fig.write_html(save_path)

### Comparison: Stations Cumulative

In [ ]:
for event, ts_fcst in ts_events.items():
    ts_fcst = ts_to_cumulativ(ts_fcst)

    start_date = ts_fcst.index[0]
    end_date = ts_fcst.index[-1]

    ts_obs = ts_stations.loc[start_date:end_date]
    ts_obs = ts_to_cumulativ(ts_obs)
    ts_obs.columns = [f"OBS {x}" for x in ts_obs.columns]

    fig = go.Figure()
    for i in range(ts_obs.shape[1]):
        color = next(colors)
        fig.add_trace(go.Scatter(x=ts_obs.index, y=ts_obs.iloc[:,i], name=ts_obs.columns[i], line=dict(color=color)))
        fig.add_trace(go.Scatter(x=ts_fcst.index, y=ts_fcst.iloc[:,i], name=ts_fcst.columns[i], line=dict(dash='dash', color=color)))

        fig.update_xaxes(title='Zeit')
        fig.update_yaxes(title="Regen kumulativ")
        fig.update_layout(title=f"{event} - Regensumme")

        save_path = save_dir / event / 'Comparison_obs-fcst_cum.html'
        fig.write_html(save_path)

### Comparison: Raster

In [ ]:
index = np.arange(0.0, 370.0, 10)
ts_diff_all = pd.DataFrame(index=index)
ts_diff_all.index.set_names('lead-time [min]', inplace=True)

for event, date in events.items():
    print(event)
    start_date = dt.strptime(date, date_format)
    end_date = start_date + td(hours=6)

    file_name = fcstname_from_date(start_date)
    file_path = dir_radarfcst / file_name

    ds_fcst = open_ds(file_path)
    ds_fcst = ds_fcst.sel(chx=slice_x, chy=slice_y)
    ds_fcst = convert_ds_units(ds_fcst, factor=mmh_mm10min, variable='RR', new_units='mm/10min')
    ds_fcst = ds_fcst.round(1)

    # Compute / plot ds_diff
    ds_diff = compute_ds_diff(ds_obs, ds_fcst, variable='RR')
    ds_diff.name = 'Regendifferenz [mm/10min]'
    save_path = save_dir / event / f"diff_{start_date.strftime('%y%m%d_%H')}.gif"
    levels = np.linspace(-4, 4, 21)
    cmap = xarray_to_gif(ds_diff, save_path=save_path, fps=1, levels=levels)

    # Compute / plot ds_diff_abs
    ds_diff_abs = abs(ds_diff)
    ds_diff_abs.name = 'Regendifferenz absolut [mm/10min]'
    save_path = save_dir / event / f"diff_abs_{start_date.strftime('%y%m%d_%H')}.gif"
    levels = np.linspace(0, 4, 21)
    xarray_to_gif(ds_diff_abs, save_path=save_path, fps=1, cmap='Reds', levels=levels)

    # Compute / plot ds_diff_rel
    ds_diff_rel = compute_ds_diff(ds_obs, ds_fcst, variable='RR', method='rel')
    ds_diff_rel = ds_diff_rel * 100
    ds_diff_rel.name = 'Regendifferenz prozent [%]'
    save_path = save_dir / event / f"diff_rel_{start_date.strftime('%y%m%d_%H')}.gif"
    # levels = np.linspace(-1, 1, 21)
    xarray_to_gif(ds_diff_rel, save_path=save_path, fps=1)


    # Compute timeseries (ts)
    ## diff
    ts_diff = sum_per_timestep(ds_diff, factor=1/(20**2))
    ts_diff_cum = ts_diff.cumsum()
    save_path = save_dir / event / 'ts_-diff.html'
    fig = hp.ply_1y([ts_diff, ts_diff_cum], xlabel='Zeit', ylabel='Regendifferenz [mm]', 
        title=f"Differenz: {event} - {date}", names=['Differenz pro Zeitschritt', 'Differenz kumulativ']);
    fig.update_layout(font=dict(size=25))
    fig.write_html(save_path)

    ## diff_abs
    ts_diff_abs = sum_per_timestep(ds_diff_abs, factor=1/(20**2))
    ts_diff_abs_cum = ts_diff_abs.cumsum()
    save_path = save_dir / event / 'ts_abs-diff.html'
    fig = hp.ply_1y([ts_diff_abs, ts_diff_abs_cum], save_path=save_path, xlabel='Zeit', ylabel='Regendifferenz [mm]',
        title=f"Absolute Differenz: {event} - {date}", names=['Differenz pro Zeitschritt', 'Differenz kumulativ']);
    fig.update_layout(font=dict(size=25))
    fig.write_html(save_path)

    # Update ts_diff_all
    time_0 = ts_diff.index[0]
    lead_time = ts_diff.index - time_0
    lead_time = lead_time.total_seconds() / 60
    ts_leadtime = pd.DataFrame(index=lead_time, data=ts_diff.iloc[:,0].cumsum().to_list())
    ts_diff_all[event] = ts_leadtime
    ts_diff_all

### Mean and Std over all Events

In [ ]:
ts_diff_all['mean'] = ts_diff_all.mean(axis='columns')
ts_diff_all['std'] = ts_diff_all.std(axis='columns')
ts_diff_all['pos-std'] = ts_diff_all[ts_diff_all>=0].std(axis='columns')
ts_diff_all['neg-std'] = ts_diff_all[ts_diff_all<=0].std(axis='columns')

fig, ax = plt.subplots()
ax.errorbar(x=ts_diff_all.index, y=ts_diff_all['mean'], yerr=[ts_diff_all['neg-std'], ts_diff_all['pos-std']],
    ecolor='red', fmt='b-x')
plt.xlabel('Vorlaufzeit [min]')
plt.ylabel('Differenz [mm]')
plt.grid(True, linestyle='--', color='grey')

ax.legend(['Mtl. kumul. Regendifferenz'])
plt.tight_layout()

save_path = save_dir / 'Regendiff_avg-cumul.png'
fig.savefig(save_path, dpi=600)
plt.close()

In [ ]:
ts_sup_inf = pd.DataFrame(ts_diff_all.iloc[:,12])
ts_sup_inf['lim-sup'] = ts_diff_all.iloc[:,12] + ts_diff_all.iloc[:,14]
ts_sup_inf['lim-inf'] = ts_diff_all.iloc[:,12] - ts_diff_all.iloc[:,15]

save_path = save_dir / 'sup-inf_data_mm.csv'
ts_sup_inf.astype(float).to_csv(save_path, sep=';')